<a href="https://colab.research.google.com/github/augustopdro/COMPLIANCE-QUALITY-ASSURANCE/blob/main/Checkpoint1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Tratamento dos dados
import numpy as np
import pandas as pd

# Modelos de Machine Learning
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

#Importando os dados
df = pd.read_csv('breast-cancer.data', sep=',', header=None)

FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
df = df.drop(7, axis=1)

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
#Reunindo informações sobre os dados numéricos
df.describe()

##Checando e confirmando que não há dados nulos na tabela. 

In [ ]:
df.isnull().sum()

In [ ]:
df = df.rename(columns={0: "Class", 1: "Age", 2: "Menopause", 3: "Tumor-size", 4: "Inv-nodes", 5: "Node-caps", 6: "Deg-malig", 8: "Breast-quad", 9: "Irradiant"})

In [ ]:
for i, age_value in enumerate(df["Age"]):
    if "-" in age_value:
        age_range = age_value.split("-")
        age_mean = (int(age_range[0]) + int(age_range[1])) // 2
        df.at[i, "Age"] = age_mean
    else:
        df.at[i, "Age"] = int(age_value)
df["Age"] = pd.to_numeric(df["Age"])

In [ ]:
for i, size_value in enumerate(df["Tumor-size"]):
    if "-" in size_value:
        size_range = size_value.split("-")
        size_mean = (int(size_range[0]) + int(size_range[1])) // 2
        df.at[i, "Tumor-size"] = size_mean
    else:
        df.at[i, "Tumor-size"] = int(size_value)
df["Tumor-size"] = pd.to_numeric(df["Tumor-size"])

In [ ]:
for i, nodes_value in enumerate(df["Inv-nodes"]):
    if "-" in nodes_value:
        nodes_range = nodes_value.split("-")
        nodes_mean = (int(nodes_range[0]) + int(nodes_range[1])) // 2
        df.at[i, "Inv-nodes"] = nodes_mean
    else:
        df.at[i, "Inv-nodes"] = int(nodes_value)

df["Inv-nodes"] = pd.to_numeric(df["Inv-nodes"])

In [ ]:
df['Irradiant'] = df['Irradiant'].replace({'yes': 1, 'no':2})
df['Breast-quad'] = df['Breast-quad'].replace({'left_low': 1, 'left_up':2, 'right_low':3, 'right_up':4, 'central': 5, '?':6})
df['Node-caps'] = df['Node-caps'].replace({'yes': 1, 'no':2, '?':3})
df['Class'] = df['Class'].replace({'no-recurrence-events': 1, 'recurrence-events':2})
df['Menopause'] = df['Menopause'].replace({'premeno': 1, 'ge40':2, 'lt40':3})

df['Irradiant'].astype(float)
df['Breast-quad'].astype(float)      #Transformando-os em números reais
df['Node-caps'].astype(float)
df['Class'].astype(float)
df['Menopause'].astype(float)


In [ ]:
df['Node-caps'].unique()

In [ ]:
# Normalizando as colunas usando o MinMaxScaler
scaler = MinMaxScaler()
df[["Age", "Tumor-size", "Inv-nodes"]] = scaler.fit_transform(df[["Age", "Tumor-size", "Inv-nodes"]])

# Padronizando as colunas usando o StandardScaler
standard_scaler = StandardScaler()
df[["Deg-malig", "Menopause", "Node-caps", "Breast-quad", "Irradiant"]] = standard_scaler.fit_transform(df[["Deg-malig", "Menopause", "Node-caps", "Breast-quad", "Irradiant"]])

In [ ]:
df.head()

In [ ]:
df.info()

## Ajustando outliers do dataset após o tratamento

In [ ]:
def ajuste_boxplot(data, colunas):
  
  # Criando uma cópia do DataFrame original
  # Dataframes atribuidos com o operador '=' compartilham os index e as informações armazenadas. Para evitar isso usamos o copy()
  dados = data.copy()
  
  for n in range(0, len(colunas)): # Percorrendo a lista das colunas
    # Recebendo os dois quartis de interesse
    quartis = np.quantile(dados[colunas[n]], [0.25,0.75])

    # Calculando os limites superiores e inferiores:
    limite_inferior = quartis[0] -1.5*(quartis[1]-quartis[0])
    limite_superior = quartis[1] +1.5*(quartis[1]-quartis[0])

    # Calculando o valor da mediana
    mediana = dados[colunas[n]].median()

    # Criando a mascara para filtrar os dados e imprimindo os valores que serão ajustados
    mask = (dados[colunas[n]] < limite_inferior) | (dados[colunas[n]] > limite_superior)
    print(f"A coluna {colunas[n]} possui {mask.sum()} outliers para serem tratados")
    
    # Substitui os valores pela mediana
    dados.loc[mask,colunas[n]] = mediana
    
    # Exibindo no console a quantidade de valores que foram tratados
    mask = (dados[colunas[n]] < limite_inferior) | (dados[colunas[n]] > limite_superior)
    print(f"A coluna {colunas[n]} possui {mask.sum()} outliers após serem tratados")

  return dados

In [ ]:
colunas_ajuste = list(df.columns)
colunas_ajuste

In [ ]:
df_corr = df.copy()

In [ ]:
df_corr = ajuste_boxplot(df, colunas_ajuste)

In [ ]:
x = df.drop(columns = 'Class')
y = df['Class']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
redeneural = MLPClassifier(verbose = True, max_iter = 200000, tol = 0.000001, activation = 'relu', #hidden-layer_sizes=(30,30,30), 
                           learning_rate_init= 0.00001)

In [ ]:
redeneural.fit(x_train, y_train)

In [ ]:
redeneural.score(x_test, y_test)